# Teste 3 Market Sizing

### Utilizando LLM para ler um demonstrativo financeiro de uma empresa e identificar o PDD 

In [1]:
import os
import glob
from dotenv import load_dotenv

# Leitura e parsing de PDF
from unstructured.partition.pdf import partition_pdf

In [2]:
import logging

arquivo_pdf = "demonstrativos/BMG.pdf"
logging.getLogger("pdfminer").setLevel(logging.ERROR) # silencia logs do pdfminer (usado pelo unstructured)
elementos = partition_pdf(filename=arquivo_pdf, languages=["por"])

for e in elementos[:250]:
    print(e.category, "->", str(e)[:200])


Title -> BMGB B3 LISTED N1
NarrativeText -> Demonstrações financeiras individuais e consolidadas em 30 de junho de 2025 e relatório do auditor independente sobre as demonstrações financeiras
Footer -> Este documento contém informação RESTRITA. Seu conteúdo é restrito às partes interessadas e previamente autorizadas dentro do contexto e do processo que estão sendo compartilhadas e utilizadas. Em cas
Title -> Índice
UncategorizedText -> Relatório da administração ........................................................................................................................................... 1 Resumo do relatório do Comitê d
Title -> RELATÓRIO DA ADMINISTRAÇÃO
NarrativeText -> A Administração do Banco Bmg S.A. e de suas Controladas (“Banco”), em conformidade com as práticas contábeis adotadas no Brasil, estabelecidas pela Lei das Sociedades por Ações e pelo Banco Central do
Title -> Banco Bmg
NarrativeText -> O maior compromisso do Banco Bmg ao longo de seus quase 100 anos de hi

In [3]:
print(f"Total de elementos extraídos: {len(elementos)}")

tipos = set(e.category for e in elementos)
print("Categorias encontradas:", tipos)


Total de elementos extraídos: 1126
Categorias encontradas: {'ListItem', 'UncategorizedText', 'NarrativeText', 'Title', 'Footer'}


In [4]:
# Filtra apenas os elementos do tipo ListItem
list_items = [e for e in elementos if e.category == "ListItem"]

print(f"Total de ListItem encontrados: {len(list_items)}")

# Pega os 5 primeiros
for i, e in enumerate(list_items[:5], 1):
    print(f"\n--- ListItem {i} ---")
    print(str(e))



Total de ListItem encontrados: 34

--- ListItem 1 ---
Mantidos para coleta dos fluxos de caixa contratuais • Mantidos para coleta dos fluxos de caixa contratuais e venda • Mantidos para negociação/outros

--- ListItem 2 ---
Ativos administrados tanto para obter fluxos de caixa constituídos apenas de pagamentos de principal e juros (Teste SPPJ), quanto para a venda;

--- ListItem 3 ---
Estes ativos são inicialmente e subsequentemente reconhecidos a valor justo mais custos de transação; e

--- ListItem 4 ---
Os ganhos e perdas não realizados (exceto perda de crédito esperada, diferenças cambiais, dividendos e receita de juros) são reconhecidos, líquidos dos impostos aplicáveis, na rubrica Resultado Abrangente Acumulado.

--- ListItem 5 ---
Ativos que não atendem os critérios de classificação das categorias anteriores; ou ativos designados no reconhecimento inicial como ao valor justo por meio do resultado para reduzir “descasamentos contábeis”;


### Vetorizando a base de dados 

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Pegar só o texto limpo
docs = [str(e) for e in elementos if str(e).strip() != ""]

print(f"Total de blocos extraídos: {len(docs)}")
print(docs[0][:300])  # exemplo do primeiro

# Criando Chunks 

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,   # até 1000 caracteres por chunk
    chunk_overlap=100  # sobreposição para manter contexto
)

chunks = text_splitter.split_text("\n".join(docs))

print(f"Total de chunks: {len(chunks)}")
print(chunks[0][:300])

Total de blocos extraídos: 1122
BMGB B3 LISTED N1
Total de chunks: 122
BMGB B3 LISTED N1
Demonstrações financeiras individuais e consolidadas em 30 de junho de 2025 e relatório do auditor independente sobre as demonstrações financeiras
Este documento contém informação RESTRITA. Seu conteúdo é restrito às partes interessadas e previamente autorizadas dentro do contexto 


### Criando embeddings 

In [13]:
import pinecone
print(pinecone.__file__)

c:\Users\Matheus Almeida\OneDrive\Documentos\Insper\Estágio\market_sizing\venv\Lib\site-packages\pinecone\__init__.py


In [14]:
# Pinecone (SDK novo usa import do módulo inteiro)
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))


# LangChain + Pinecone + OpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Inicializa o cliente
load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Cria o índice (só precisa fazer 1 vez, se não existir ainda)
index_name = "meu-indice"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # precisa bater com a dimensão do embedding
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Conecta ao índice
index = pc.Index(index_name)

# Carrega os chunks para o Pinecone
vectorstore = PineconeVectorStore.from_texts(
    texts=chunks,
    embedding=embeddings,
    index_name=index_name
)

print("Base vetorial criada no Pinecone com sucesso!")

Base vetorial criada no Pinecone com sucesso!


In [15]:
query = "Qual o valor da Provisão para Créditos de Liquidação Duvidosa do ano de 2024?"
resultados = vectorstore.similarity_search(query, k=3)

for r in resultados:
    print("\n--- Trecho encontrado ---")
    print(r.page_content)


--- Trecho encontrado ---
Os créditos tributários relacionados as adições temporárias referem-se, principalmente, a Provisões para causas fiscais e previdenciárias discutidos em âmbito judicial ou administrativo, provisões trabalhistas e cíveis, cuja realização depende do encerramento dos respectivos processos, e provisão para crédito de liquidação duvidosa cuja realização depende dos critérios de dedutibilidade nos termos da Lei nº 14.467/2022.
Os estudos técnicos elaborados em junho de 2025 demonstram a capacidade da Instituição de geração de lucros tributáveis suficientes para compensar os créditos tributários existentes.
(b) demonstrada como segue:
Expectativa de realização dos créditos tributários no semestre findo em 30 de junho de 2025 pode ser
AnoConsolidadoBanco2025 420.497 401.230 2026 522.267 465.112 2027 597.843 482.394 2028 457.768 403.085 2029 512.455 455.444 Após 2029 2.425.843 2.340.250 Total 4.936.673 4.547.515
(c)
Obrigações fiscais diferidas - imposto de renda e con